In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

#image = mpimg.imread('test_images/test6.jpg')


In [2]:
import glob

def cc():
    # Camera Calibration
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

    objpoints = [] #3D
    imgpoints = [] #2D

    images = glob.glob('camera_cal/calibration*.jpg')
        
    for path in images:
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
        
    return objpoints, imgpoints

objpoints, imgpoints = cc()

In [3]:
def undistort(img, objpoints, imgpoints):
    img_size = (img.shape[1], img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

'''
dst_im = undistort(image, objpoints, imgpoints)

f, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
ax1.set_title('Source Image')
ax1.imshow(image)
ax2.set_title('Undistorted Image')
ax2.imshow(dst_im)
plt.show()
'''

"\ndst_im = undistort(image, objpoints, imgpoints)\n\nf, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))\nax1.set_title('Source Image')\nax1.imshow(image)\nax2.set_title('Undistorted Image')\nax2.imshow(dst_im)\nplt.show()\n"

In [4]:
def warp(img, unwrap=False):
    img_size = (img.shape[1], img.shape[0])
    
    src = np.float32(
        [[765,450], # right top
         [1050,650], # right bottom
         [330,650], # left bottom
         [560,450]]) #left top
    
    dst = np.float32(
        [[1200,0], #right top
         [1200,700], # right bottom
         [200,700], # left bottom
         [200,0]]) 
    if unwrap:
        M = cv2.getPerspectiveTransform(dst, src)
    else:
        M = cv2.getPerspectiveTransform(src,dst)
    
    warped = cv2.warpPerspective(img, M, img_size, flags= cv2.INTER_LINEAR)
    
    return warped
'''
warped_im = warp(dst_im)

f, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
ax1.set_title('Pre-Wrap Image')
ax1.imshow(dst_im)
ax1.plot(765,450,'.')
ax1.plot(1050,650,'.')
ax1.plot(330,650,'.')
ax1.plot(600,450,'.')
ax2.set_title('Warped Image')
ax2.imshow(warped_im)
plt.show()
'''

"\nwarped_im = warp(dst_im)\n\nf, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))\nax1.set_title('Pre-Wrap Image')\nax1.imshow(dst_im)\nax1.plot(765,450,'.')\nax1.plot(1050,650,'.')\nax1.plot(330,650,'.')\nax1.plot(600,450,'.')\nax2.set_title('Warped Image')\nax2.imshow(warped_im)\nplt.show()\n"

In [5]:
def img_preprocess(img):
    s_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:,:,2]
    l_channel = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)[:,:,0]
    b_channel = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)[:,:,2]

    # Threshold color channel
    s_min = 170
    s_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_min) & (s_channel <= s_max)] = 1

    b_min = 150
    b_max = 200
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_min) & (b_channel <= b_max)] = 1

    l_min = 200
    l_max = 255
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_min) & (l_channel <= l_max)] = 1

    combined_binary = np.zeros_like(s_binary)
    combined_binary[(s_binary == 1) | (l_binary == 1) | (b_binary == 1)] = 1
    
    return combined_binary

In [6]:
def lane_line(img, image):
    binary_warped = img
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    src = np.float32(
        [[765,450], # right top
         [1050,650], # right bottom
         [330,650], # left bottom
         [560,450]]) #left top

    dst = np.float32(
        [[1200,0], #right top
         [1200,700], # right bottom
         [200,700], # left bottom
         [200,0]])

    Minv = cv2.getPerspectiveTransform(dst, src)

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    return result

In [7]:
def vid_process(img):
    dst_im = undistort(img, objpoints, imgpoints)
    warpped = warp(dst_im)
    preprocessed = img_preprocess(warpped)
    result = lane_line(preprocessed, dst_im)
    return result

In [ ]:
from moviepy.editor import VideoFileClip

clip = VideoFileClip("project_video.mp4")
white_clip = clip.fl_image(vid_process)
white_clip.write_videofile('result.mp4', audio=False)

[MoviePy] >>>> Building video result.mp4
[MoviePy] Writing video result.mp4


 89%|████████▉ | 1123/1261 [1:00:04<07:25,  3.23s/it]